NeuralMMOに慣れることも兼ねて初めに取り組む内容

20221011時点での越田の大きな疑問としては以下の二つ。<br>
- エージェントはチームを組んでいるようだが、そのチーム全体を実装するのか？
- 各特徴量は連続値と離散値を持っていて、離散値は連続値をサンプリングしたもののように思える。そして離散値を捨ててしまっても構わないとされているが、それではなんのために離散値は保持されているのか？

# **Tutorial on NMMO's observation and action space**
This is a tutorial to show the observation space and action space of Neural MMO's environment.

## **Installation**

See https://gitlab.aicrowd.com/neural-mmo/neurips2022-nmmo-starter-kit

In [1]:
# install neurips competition setup and nmmo
!pip install --ignore-requires-python openskill==0.2.0a0
!pip install git+http://gitlab.aicrowd.com/neural-mmo/neurips2022-nmmo.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 42 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning http://gitlab.aicrowd.com/neural-mmo/neurips2022-nmmo.git to /tmp/pip-req-build-f7hp5v1y
  Running command git clone -q http://gitlab.aicrowd.com/neural-mmo/neurips2022-nmmo.git /tmp/pip-req-build-f7hp5v1y
     |████████████████████████████████| 630 kB 2.1 MB/s 
     |████████████████████████████████| 58 kB 5.4 MB/s 
     |████████████████████████████████| 50 kB 6.2 MB/s 
     |████████████████████████████████| 87 kB 6.2 MB/s 
     |████████████████████████████████| 389 kB 44.1 MB/s 
     |████████████████████████████████| 3.1 MB 51.8 MB/s 
     |████████████████████████████████| 134 kB 44.4 MB/s 
     |████████████████████████████████| 3.3 MB 32.4 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████

After installation, we can establish a demo environment to show how NMMO works.



In [2]:
import nmmo
from nmmo import config

class TestConfig(config.Medium, config.AllGameSystems):
    pass

conf = TestConfig()
env = nmmo.Env( conf )

100%|██████████| 1/1 [00:00<00:00, 11.02it/s]

Generating 1 maps


The env setting for this competition:

- Map of 128 * 128 grids
- 128 players (16 teams * 8 players/team)
- All players spawn(生まれる) concurrently(並列処理的に) when game starts

If you want to see the detailed configuration：

In [3]:
for attr in dir(config):
    if not attr.startswith( '__' ):
        print( f'{attr}: {getattr( config, attr )} ' )


AllGameSystems: <class 'nmmo.core.config.AllGameSystems'> 
Combat: <class 'nmmo.core.config.Combat'> 
Communication: <class 'nmmo.core.config.Communication'> 
Config: <class 'nmmo.core.config.Config'> 
Default: <class 'nmmo.core.config.Default'> 
Equipment: <class 'nmmo.core.config.Equipment'> 
Exchange: <class 'nmmo.core.config.Exchange'> 
Item: <class 'nmmo.core.config.Item'> 
Large: <class 'nmmo.core.config.Large'> 
Medium: <class 'nmmo.core.config.Medium'> 
NPC: <class 'nmmo.core.config.NPC'> 
Profession: <class 'nmmo.core.config.Profession'> 
Progression: <class 'nmmo.core.config.Progression'> 
Resource: <class 'nmmo.core.config.Resource'> 
Small: <class 'nmmo.core.config.Small'> 
T: <function set_trace at 0x7f8b111248c0> 
Template: <class 'nmmo.core.config.Template'> 
Terrain: <class 'nmmo.core.config.Terrain'> 
material: <module 'nmmo.lib.material' from '/usr/local/lib/python3.7/dist-packages/nmmo/lib/material.py'> 
nmmo: <module 'nmmo' from '/usr/local/lib/python3.7/dist-packag

## **Observation Space**
You can get the observation space via env.observation_space(agent) but the output is really verbose(冗長な).

In [4]:
env.observation_space(agent=1)

Dict('Entity': Dict('Continuous': Box(-1048576.0, 1048576.0, (100, 24), float32), 'Discrete': Box(0, 4096, (100, 5), int32), 'N': Box(0, 100, (1,), int32)), 'Item': Dict('Continuous': Box(-1048576.0, 1048576.0, (170, 16), float32), 'Discrete': Box(0, 4096, (170, 3), int32), 'N': Box(0, 170, (1,), int32)), 'Market': Dict('Continuous': Box(-1048576.0, 1048576.0, (170, 16), float32), 'Discrete': Box(0, 4096, (170, 3), int32), 'N': Box(0, 170, (1,), int32)), 'Tile': Dict('Continuous': Box(-1048576.0, 1048576.0, (225, 4), float32), 'Discrete': Box(0, 4096, (225, 3), int32), 'N': Box(0, 15, (1,), int32)))

It is hard to understand, so the obs will be explained in detail in the following chapter.

In [5]:
obs = env.reset()
print(obs.keys())

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

The env returns obs in a dict and the dict keys are the agent's id from 1 to 128.

And let's take a look at the obs single agent receives:

In [6]:
obs[1].keys()

dict_keys(['Entity', 'Tile', 'Item', 'Market'])

## The agent's obs consists of two parts:
↑two parts?four partsじゃないの？
- Entity: the information of yourself, other players and npcs.
- Tile: the information of local map with 15x15 size.
- Item: the information of weapon, tool, comsummer that the Entity equiped
- Market: the information (in selling goods) of global market

## Entity Information

"Entity"属性は自分自身か、他のプレイヤーか、NPCのいずれかの情報を持っている。

In [7]:
obs[1]['Entity'].keys()

dict_keys(['Continuous', 'Discrete', 'N'])

In [8]:
obs[1]['Entity']['Continuous'].shape

(100, 24)

In [9]:
obs[1]['Entity']['Discrete'].shape

(100, 5)

In [10]:
obs[1]['Entity']['N']

array([9], dtype=int32)

The entity information is a dictionary with the following keys

- Continuous: the continuous features, a 2d ndarray with shape 100*24.
    - The first dimention 100 is the max number of agents that can be observed, and is controlled by config.N_AGENT_OBS.
    - The second dimention 24 is the number of feature columns and the meaning of each column will be explained in detail.
- Discrete: the discrete features, a 2d ndarray with shape 100*5.
    - The first dimention 100 is the max number of agents that can be observed, and is controlled by config.N_AGENT_OBS.
    - The second dimention 5 is the number of feature columns.
    - Notice that the discrete information is duplicate of (a part of) the continuous information, which means you can simply drop the discrete information.
- N: the number of agents observed (including yourself) in current vision.

## Tile Information

"Tile"属性は、Entityの存在するローカルの位置情報を保持している。

In [11]:
obs[1]['Tile'].keys()

dict_keys(['Continuous', 'Discrete', 'N'])

In [12]:
obs[1]['Tile']['Continuous'].shape

(225, 4)

In [13]:
obs[1]['Tile']['Discrete'].shape

(225, 3)

In [14]:
obs[1]['Tile']['N']

array([15], dtype=int32)

The tile information is a dictionary with the following keys

- Continuous: the continuous features, a 2d ndarray with shape 225*4.
    - The first dimention 225 is the number of tiles within agent's vision, which is controlled by config.NSTIM. When config.NSTIM=7 by default, the number of tiles is (2*7+1)^2 = 225.
    - The second dimention 4 is the number of feature columns and the meaning of each column will be explained in detail.
- Discrete: the discrete features, a 2d ndarray with shape 225*3.
    - The first dimention 225 is the number of tiles within agent's vision, which is controlled by config.NSTIM.
    - The second dimention 3 is the number of feature columns.
    - Notice that the discrete information is also duplicate of (a part of) the continuous information, which means you can simply drop the discrete information.
- N: the number of distinct tile observations (fixed)
## Item Information

"Item"属性は、Entityのもつアイテム情報を保持している。例えば、武器や食糧情報など。

In [15]:
obs[1]['Item'].keys()

dict_keys(['Continuous', 'Discrete', 'N'])

In [16]:
obs[1]['Item']['Continuous'].shape

(170, 16)

In [17]:
obs[1]['Item']['Discrete'].shape

(170, 3)

In [18]:
obs[1]['Item']['N']

array([1], dtype=int32)

The item information is a dictionary with the following keys

- Continuous: the continuous features, a 2d ndarray with shape 170*16.
    - The first dimention 170 is the number of items, which is controlled by config.NPC_LEVEL_MAX. When config.NPC_LEVEL_MAX=10 by default, the number of items is 17*10 = 170.
    - The second dimention 16 is the number of feature columns and the meaning of each column will be explained in detail.
- Discrete: the discrete features, a 2d ndarray with shape 170*3.
    - The first dimention 170 is the number of items, which is controlled by config.NPC_LEVEL_MAX. When config.NPC_LEVEL_MAX=10 by default, the number of items is 17*10 = 170.
    - The second dimention 3 is the number of feature columns.
    - Notice that the discrete information is also duplicate of (a part of) the continuous information, which means you can simply drop the discrete information.
- N: the number of distinct item observations (fixed)

## Market Information

"Market"属性は、市場での売買情報や商品情報を保持している。

In [19]:
obs[1]['Market'].keys()

dict_keys(['Continuous', 'Discrete', 'N'])

In [20]:
obs[1]['Market']['Continuous'].shape

(170, 16)

In [21]:
obs[1]['Market']['Discrete'].shape

(170, 3)

In [22]:
obs[1]['Market']['N']

array([0], dtype=int32)

The market information is a dictionary with the following keys

- Continuous: the continuous features, a 2d ndarray with shape 170*16.
    - The first dimention 170 is the max number of items in market, which is controlled by config.NPC_LEVEL_MAX. When config.NPC_LEVEL_MAX=10 by default, the number of items is 17*10 = 170.
    - The second dimention 16 is the number of feature columns and the meaning of each column will be explained in detail.
- Discrete: the discrete features, a 2d ndarray with shape 170*3.
    - The first dimention 170 is the number of items, which is controlled by config.NPC_LEVEL_MAX. When config.NPC_LEVEL_MAX=10 by default, the number of items is 17*10 = 170.
    - The second dimention 3 is the number of feature columns.
    - Notice that the discrete information is also duplicate of (a part of) the continuous information, which means you can simply drop the discrete information.
- N: the number of distinct item observations (fixed)

## Feature Columns

| Raw Features      | index1 | index2 |  Dimension | Type   | attr            |
|-----------|--------|--------|----------------------|--------|-----------------|
| Obs (raw obs) | Entity (100*24) | 0      | 1  | scalar | Mask            |
|           |        | 1      | 1  | scalar | "ID"            |
|           |        | 2      | 1  | scalar | AttackerID      |
|           |        | 3      | 1  | scalar | Level           |
|           |        | 4      | 1  | scalar | ItemLevel       |
|           |        | 5      | 1  | scalar | Comm            |
|           |        | 6      | 1  | scalar | Population      |
|           |        | 7      | 1  | scalar | R               |
|           |        | 8      | 1  | scalar | C               |
|           |        | 9      | 1  | scalar | Damage          |
|           |        | 10     | 1  | scalar | TimeAlive       |
|           |        | 11      | 1  | scalar | Freeze (deprecated)          |
|           |        | 12     | 1  | scalar | Gold            |
|           |        | 13     | 1  | scalar | Health          |
|           |        | 14     | 1  | scalar | Food            |
|           |        | 15     | 1  | scalar | Water           |
|           |        | 16     | 1  | scalar | Melee Level           |
|           |        | 17     | 1  | scalar | Range Level           |
|           |        | 18     | 1  | scalar | Mage Level            |
|           |        | 19     | 1  | scalar | Fishing Level         |
|           |        | 20     | 1  | scalar | Herbalism Level       |
|           |        | 21     | 1  | scalar | Prospecting Level     |
|           |        | 22     | 1  | scalar | Carving Level         |
|           |        | 23     | 1  | scalar | Alchemy Level         |
|           | Tile (225*4)   | 0   | 1  | scalar | NEnts           |
|           |        | 1      | 1  | scalar | Index           |
|           |        | 2      | 1  | scalar | R               |
|           |        | 3      | 1  | scalar | C               |
|           | Item (170*16)   | 0   | 1  | scalar | "ID"            |
|           |        | 1      | 1  | scalar | Index           |
|           |        | 2      | 1  | scalar | Level           |
|           |        | 3      | 1  | scalar | Capacity        |
|           |        | 4      | 1  | scalar | Quantity        |
|           |        | 5      | 1  | scalar | Tradable        |
|           |        | 6      | 1  | scalar | MeleeAttack     |
|           |        | 7      | 1  | scalar | RangeAttack     |
|           |        | 8      | 1  | scalar | MageAttack      |
|           |        | 9      | 1  | scalar | MeleeDefense    |
|           |        | 10     | 1  | scalar | RangeDefense    |
|           |        | 11     | 1  | scalar | MageDefense     |
|           |        | 12     | 1  | scalar | HealthRestore   |
|           |        | 13     | 1  | scalar | ResourceRestore |
|           |        | 14    | 1  | scalar | Price           |
|           |        | 15     | 1  | scalar | Equipped        |
|           | Market (170*16) | 0    | 1  | scalar | "ID"            |
|           |        | 1     | 1  | scalar | Index           |
|           |        | 2      | 1  | scalar | Level           |
|           |        | 3     | 1  | scalar | Capacity        |
|           |        | 4      | 1  | scalar | Quantity        |
|           |        | 5     | 1  | scalar | Tradable        |
|           |        | 6      | 1  | scalar | MeleeAttack     |
|           |        | 7      | 1  | scalar | RangeAttack     |
|           |        | 8     | 1  | scalar | MageAttack      |
|           |        | 9      | 1  | scalar | MeleeDefense    |
|           |        | 10     | 1  | scalar | RangeDefense    |
|           |        | 11     | 1  | scalar | MageDefense     |
|           |        | 12     | 1  | scalar | HealthRestore   |
|           |        | 13    | 1  | scalar | ResourceRestore |
|           |        | 14    | 1  | scalar | Price           |
|           |        | 15     | 1  | scalar | Equipped        |


## **Action Space**

エージェントの行動指針

In [23]:
from nmmo.io import action
act_space = env.action_space(agent=1)
print("Action space:")
print("*"*2,action.Attack,": ",act_space[action.Attack])
print("-"*8,action.Style,": ",act_space[action.Attack][action.Style])
print("-"*8,action.Target,": ",act_space[action.Attack][action.Target])
print("*"*2,action.Move,": ",act_space[action.Move])
print("-"*8,action.Direction,act_space[action.Move][action.Direction])
print("*"*2,action.Buy,": ",act_space[action.Buy])
print("-"*8,action.Item, act_space[action.Buy][action.Item])
print("*"*2,action.Sell,": ",act_space[action.Sell])
print("-"*8,action.Item, act_space[action.Sell][action.Item])
print("-"*8,action.Price, act_space[action.Sell][action.Price])
print("*"*2,action.Use,": ",act_space[action.Use])
print("-"*8,action.Item, act_space[action.Use][action.Item])
print("*"*2,action.Comm,": ",act_space[action.Comm])
print("-"*8,action.Token, act_space[action.Comm][action.Token])

Action space:
** <class 'nmmo.io.action.Attack'> :  Dict(<class 'nmmo.io.action.Style'>: Discrete(3), <class 'nmmo.io.action.Target'>: Discrete(100))
-------- <class 'nmmo.io.action.Style'> :  Discrete(3)
-------- <class 'nmmo.io.action.Target'> :  Discrete(100)
** <class 'nmmo.io.action.Move'> :  Dict(<class 'nmmo.io.action.Direction'>: Discrete(4))
-------- <class 'nmmo.io.action.Direction'> Discrete(4)
** <class 'nmmo.io.action.Buy'> :  Dict(<class 'nmmo.io.action.Item'>: Discrete(170))
-------- <class 'nmmo.io.action.Item'> Discrete(170)
** <class 'nmmo.io.action.Sell'> :  Dict(<class 'nmmo.io.action.Item'>: Discrete(170), <class 'nmmo.io.action.Price'>: Discrete(100))
-------- <class 'nmmo.io.action.Item'> Discrete(170)
-------- <class 'nmmo.io.action.Price'> Discrete(100)
** <class 'nmmo.io.action.Use'> :  Dict(<class 'nmmo.io.action.Item'>: Discrete(170))
-------- <class 'nmmo.io.action.Item'> Discrete(170)
** <class 'nmmo.io.action.Comm'> :  Dict(<class 'nmmo.io.action.Token'>:

As shown above, the action space is presented as a nested dictionary. And the keys are the classes from nmmo.io.action.

The agent can perform 6 actions at the same timestep:

1. Attack: you can attack an entity (include npc and other players) within your vision.The action can be empty.(If you don't send this action, the agent will not attack anyone.)

    - Target: Choose the target the agent attack.
    - Style: Choose the style the agent use to attack.

2. Move: you can move in the four directions. The action can be empty.(If you don't send this action, the agent will stay.)

    - Direction: Choose the direction the agent moves.

3. Buy: you can buy some items from the glbal market. (If you don't send this action, the agent will not buy any items)

    - Item: Choose items that the agent need in global market to buy.

4.  Sell: you can put some items on the glbal market with preset price. (If you don't send this action, the agent will not sell any items)

    - Item: Choose the items that the agent not need to sell.
    - Price: Choose a preset price to sell the Item.

5. Use: you can use the consumer item to restrore healthy, food or water. (If you don't send this action, the agent will not use any items)

    - Item: Choose the consumerable items to restore healthy, food or water.
    
6. Comm: you can show other agents that items you need

    - Token: represent the items you need

And you should return actions like this:

エージェントの行動指針を定義

In [24]:
from nmmo.io import action
actions = {
  1: {
    action.Attack:{
      action.Style: 0,
      action.Target: 3
      },
    action.Move: {
      action.Direction: 1
    },
    action.Buy: {
      action.Item: 4
    },
    action.Sell: {
      action.Item: 2,
      action.Price: 5
    },
    action.Use: {
      action.Item: 4
    },
    action.Comm: {
      action.Token: 0
    }
  },
  2: {
    action.Attack:{
      action.Style: 0,
      action.Target: 3
      },
    action.Move: {
      action.Direction: 1
    },
    action.Buy: {
      action.Item: 4
    },
    action.Sell: {
      action.Item: 2,
      action.Price: 5
    },
    action.Use: {
      action.Item: 4
    },
    action.Comm: {
      action.Token: 0
    }
  },
  ...
}

SyntaxError: ignored

# **Team-Based Env**

With game setting of the competition, it would be better to make the input and output based on team. Therefore, users can focus on training teams, and do not need to care about how to seperate players into teams manully.

The TeamBasedEnv is almost the same as the nmmo.Env except for the reset() and step() methods which handle the team-player mapping automatically.

TeamBasedEnv is used in evaluation, and it may be used in training too.

In [ ]:
from neurips2022nmmo import CompetitionConfig, TeamBasedEnv

env = TeamBasedEnv(CompetitionConfig())

# input
actions_by_team = {
    # actions of team0
    0: {
        0: action_of_player0_in_team0,
        1: action_of_player1_in_team0,
        ...
        7: action_of_player7_in_team0,
    },
    # actions of team1
    1: {
        0: action_of_player0_in_team1,
        ...
    },
    ...
    # actions of team15
    15: {
        ...
        7: action_of_player7_in_team15
    },
}

(   
    observations_by_team, 
    rewards_by_team, 
    dones_by_team, 
    infos_by_team,
) = env.step(actions_by_team)

print(observations_by_team)

...

{
    # observations of team0
    0: {
        0: obs_of_player0_in_team0,
        1: obs_of_player1_in_team0,
        ...
        7: obs_of_player7_in_team0,
        'stat': current statistical data,
    },
    # observations of team1
    1: {
        0: obs_of_player0_in_team1,
        ...
    },
    ...
    # observations of team15
    15: {
        ...
        7: obs_of_player7_in_team15
    },
}